### Caching
Storing results in a hashtable in order to save for later
Frequent things do not need to be re computed

- browser, database, Google/IR

#### Cache Invalidation
When shoud we delete things. Gnerally by guessing

### Hit rate
Look up N things, find k things hite rate=k/N
    ~= How many thigns found/ how many tiems tried
    
### Optimize
#### Just turn the fixed size hap map inot unfixed one
- Linked hash map ->  can turn easily into FIFO 
- FIFO -> When need more space, kick put the oldest
     - May or may not be good depending on the list
     - Have to make te data random. Consider [A B C D A B C D] and a FIFO list of size 3
         - miss rate 100%
     
#### Random
- Fair

#### Least Recently Used (LRU)
- Close the tab haven't touched for the most time

#### Least Frequently Used (LFU)
- Count the times a slot has a hit and always kick out the least popular




In [10]:
import urllib.request
# https://www.gutenberg.org/ebooks/42671
DATA_URL = "http://www.gutenberg.org/files/1342/1342-0.txt"

data = []

for line in urllib.request.urlopen(DATA_URL):
    data.extend([w.decode('utf-8-sig') for w in line.strip().lower().split()])
len(data), ' '.join(data[5:11])

(124592, 'pride and prejudice, by jane austen')

In [11]:
import time
from tqdm import tqdm_notebook as tqdm
import random

# This simulated slot in a cache keeps track of timing and usages.
class CacheSlot(object):
  def __init__(self):
    self.used = False
    self.get_time = time.time()
    self.replaced_time = time.time()
    self.uses = 0
    self.key = None
  def replace(self, key):
    self.key = key
    self.replaced_time = time.time()
    self.uses = 0
  def get(self):
    self.get_time = time.time()
    self.uses += 1

# Configure the number of buckets available when you make a new WordCache
class WordCache(object):
  def __init__(self, num_buckets = 1000, algorithm="LRU"):
    self.algorithm = algorithm.lower()
    self.num_buckets = num_buckets
    self.empty_spots = [CacheSlot() for _ in range(num_buckets)]
    self.used_spots = []
    self.misses = 0
    self.hits = 0
    self.accesses = 0
    self.mapping = {}
  def lookup(self, key):
    self.accesses += 1
    if key in self.mapping:
      slot = self.mapping[key]
      if slot.key == key:
        slot.get()
        self.hits += 1
        return True
    self.insert(key)
    self.misses += 1
    return False
  
  
  def find_lru(self):
    ## Sort by the get_time of the latest algorithm:
    (_lru_time, lru_idx) = min((s.get_time, i) for (i, s) in enumerate(self.used_spots))
    return lru_idx
  
  def find_lfu(self):
    ## Sort by the get_time of the latest algorithm:
    (_lfu_time, lfu_idx) = min((s.uses, i) for (i, s) in enumerate(self.used_spots))
    return lfu_idx
  
  def find_random(self):
    ## return a random one from all used spots
    return random.randint(0, len(self.used_spots)-1)  
  
  def find_fifo(self):
    ## return the oldest used spot
    return min((s.replaced_time, i) for (i, s) in enumerate(self.used_spots))[1]
  
  def insert(self, key):
    slot = None
    if len(self.empty_spots) != 0:
      slot = self.empty_spots.pop()
      self.used_spots.append(slot)
    else:
      if self.algorithm == "lru":
        slot = self.used_spots[self.find_lru()]
      elif self.algorithm == "random":
        slot = self.used_spots[self.find_random()]
      elif self.algorithm == "fifo":
        slot = self.used_spots[self.find_fifo()]
      elif self.algorithm == "lfu":
        slot = self.used_spots[self.find_lfu()]
      else:
        raise ValueError(algorithm)
    if slot.key is not None:
      del self.mapping[slot.key]
    self.mapping[key] = slot
    slot.replace(key)
    
# Test on a small set of data.
cache = WordCache()

# Don't use the whole set of data for speed (test that there are no python errors in a print-friendly way)
for w in tqdm(data[:100]):
  cache.lookup(w)

# The two things we care about are hit-ratio and miss-ratio.
print("MISS RATIO: %1.3f" % (cache.misses / cache.accesses))
print("HIT RATIO: %1.3f" % (cache.hits / cache.accesses))

ModuleNotFoundError: No module named 'tqdm'

In [12]:
import matplotlib.pyplot as plt

In [13]:
# How big should the cache be?
N = 1000

#test on the full set of data:
cache = WordCache(num_buckets=N, algorithm="fifo")

for w in tqdm(data):
  cache.lookup(w)

print("HIT RATIO: %1.3f" % (cache.hits / cache.accesses))

NameError: name 'WordCache' is not defined

In [14]:
# Cheating experiment: look at the frequency of the dataset and then choose the most popular.

from collections import Counter
frequencies = Counter(data)

def hit_ratio(N):
  topN = sorted([(v,k) for (k,v) in frequencies.items()], reverse=True)[:N]
  static_by_frequency = dict((k,v) for (v,k) in topN)

  # test on the full set of data (warm cache)
  hits = 0
  accesses = len(data)
  misses = 0
  for w in tqdm(data):
    if w in static_by_frequency:
      hits+=1
    else:
      misses += 1
  return hits/accesses

print("HIT RATIO: %1.3f" % (hits / accesses))
Ns = [1,10,500,1000,5000]
ratios = [hit_ratio(x) for x in Ns]
plt.plot(Ns,ratios)

NameError: name 'hits' is not defined